In [1]:
import boto3
import pandas as pd
import time
import s3fs
from io import StringIO #BytesIO

In [2]:
AWS_ACCESS_KEY = ""
AWS_SECRET_KEY = ""
AWS_REGION = "us-east-1"
SCHEMA_NAME = "covid_dataset"
S3_STAGING_DIR = "s3://pngo-covid-athena-output/output/"
S3_BUCKET_NAME = "pngo-covid-athena-output"
S3_OUTPUT_DIR= "output"

In [3]:
athena_client = boto3.client(
    "athena",
    aws_access_key_id = AWS_ACCESS_KEY,
    aws_secret_access_key = AWS_SECRET_KEY,
    region_name = AWS_REGION,
)

In [4]:
Dict = {}

#download function 
def download_and_load_query_results (
        client:boto3.client, 
        query_response:Dict
)-> pd.DataFrame:
    while True:
        try: 
            # client.get_query_execution(QueryExecutionId = query_response["QueryExecutionId"])
            client.get_query_results(QueryExecutionId = query_response["QueryExecutionId"])
            break
        except Exception as err:
            if "not yet finished" in str(err):
                time.sleep(0.001)
            else:
                raise err
            
    temp_file_location: str = "athena_query_results.csv"
    s3_client = boto3.client(
        "s3",
        aws_access_key_id = AWS_ACCESS_KEY,
        aws_secret_access_key = AWS_SECRET_KEY,
        region_name = AWS_REGION,
    )

    s3_client.download_file(
        S3_BUCKET_NAME,
        f"{S3_OUTPUT_DIR}/{query_response['QueryExecutionId']}.csv",
        temp_file_location,
    )
    return pd.read_csv(temp_file_location) 

In [5]:
#enigma_jhud download

response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM enigma_jhud limit 1000;",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

engima_jhud = download_and_load_query_results(athena_client, response)
engima_jhud

,fips,admin2,province_state,country_region,last_update,latitude,longitude,confirmed,deaths,recovered,active,combined_key,partition_0
0,NaN,NaN,Anhui,China,2020-01-22T17:00:00,31.826,117.226,1.0,NaN,NaN,NaN,"""Anhui",csv
1,NaN,NaN,Beijing,China,2020-01-22T17:00:00,40.182,116.414,14.0,NaN,NaN,NaN,"""Beijing",csv
2,NaN,NaN,Chongqing,China,2020-01-22T17:00:00,30.057,107.874,6.0,NaN,NaN,NaN,"""Chongqing",csv
3,NaN,NaN,Fujian,China,2020-01-22T17:00:00,26.079,117.987,1.0,NaN,NaN,NaN,"""Fujian",csv
4,NaN,NaN,Gansu,China,2020-01-22T17:00:00,36.061,103.834,NaN,NaN,NaN,NaN,"""Gansu",csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,NaN,NaN,Jilin,China,2020-02-12T14:13:08,43.666,126.192,83.0,1.0,22.0,NaN,"""Jilin",csv
996,NaN,NaN,Inner Mongolia,China,2020-02-12T11:13:05,44.093,113.945,60.0,0.0,6.0,NaN,"""Inner Mongolia",csv
997,NaN,NaN,Xinjiang,China,2020-02-12T01:43:02,41.113,85.240,59.0,0.0,3.0,NaN,"""Xinjiang",csv
998,NaN,NaN,Ningxia,China,2020-02-12T11:13:05,37.269,106.165,58.0,0.0,24.0,NaN,"""Ningxia",csv


In [6]:
#us_county download

response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM us_county limit 1000;",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

us_county = download_and_load_query_results(athena_client, response)

In [7]:
#us_states download

response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM us_states limit 1000;",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

us_states = download_and_load_query_results(athena_client, response)

In [8]:
#countrycode download

response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM countrycode limit 1000;",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

countrycode = download_and_load_query_results(athena_client, response)

In [9]:
#countypopulation download

response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM countypopulation limit 1000;",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

countypopulation = download_and_load_query_results(athena_client, response)

In [10]:
#rearc_usa_hospital_beds download

response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM rearc_usa_hospital_beds limit 1000;",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

# rearc_usa_hospital_beds = download_and_load_query_results(athena_client, response)

In [11]:
#state_abv download

response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM state_abv limit 1000;",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

state_abv = download_and_load_query_results(athena_client, response)
state_abv.head()

,col0,col1
0,State,Abbreviation
1,Alabama,AL
2,Alaska,AK
3,Arizona,AZ
4,Arkansas,AR


In [12]:
#states_daily download

response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM states_daily limit 1000;",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

states_daily = download_and_load_query_results(athena_client, response)
states_daily.head()

,date,state,positive,probablecases,negative,pending,totaltestresultssource,totaltestresults,hospitalizedcurrently,hospitalizedcumulative,...,dataqualitygrade,deathincrease,hospitalizedincrease,hash,commercialscore,negativeregularscore,negativescore,positivescore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [13]:
#us_daily download

response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM us_daily limit 1000;",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

us_daily = download_and_load_query_results(athena_client, response)

In [14]:
#us_total_latest download

response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM us_total_latest limit 1000;",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration = {
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

us_total_latest = download_and_load_query_results(athena_client, response)

In [15]:
# Cleaning data

new_header = state_abv.iloc[0]
state_abv = state_abv[1:]
state_abv.columns = new_header
state_abv.head()


,State,Abbreviation
1,Alabama,AL
2,Alaska,AK
3,Arizona,AZ
4,Arkansas,AR
5,California,CA


In [16]:
## Extract columns from staging tables

# Fact table

factCovid_1 = engima_jhud[[
    'fips','province_state','country_region','confirmed','deaths','recovered','active'
]]
factCovid_1


,fips,province_state,country_region,confirmed,deaths,recovered,active
0,NaN,Anhui,China,1.0,NaN,NaN,NaN
1,NaN,Beijing,China,14.0,NaN,NaN,NaN
2,NaN,Chongqing,China,6.0,NaN,NaN,NaN
3,NaN,Fujian,China,1.0,NaN,NaN,NaN
4,NaN,Gansu,China,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
995,NaN,Jilin,China,83.0,1.0,22.0,NaN
996,NaN,Inner Mongolia,China,60.0,0.0,6.0,NaN
997,NaN,Xinjiang,China,59.0,0.0,3.0,NaN
998,NaN,Ningxia,China,58.0,0.0,24.0,NaN


In [17]:
factCovid_2 = states_daily[[
    'fips','date','positive','negative','hospitalizedcurrently','hospitalized','hospitalizeddischarged'
]]
factCovid_2

,fips,date,positive,negative,hospitalizedcurrently,hospitalized,hospitalizeddischarged
0,2,20210307,56886,NaN,33.0,1293.0,NaN
1,1,20210307,499819,1931711.0,494.0,45976.0,NaN
2,5,20210307,324818,2480716.0,335.0,14926.0,NaN
3,60,20210307,0,2140.0,NaN,NaN,NaN
4,4,20210307,826454,3073010.0,963.0,57907.0,118932.0
...,...,...,...,...,...,...,...
995,44,20210218,122859,654308.0,180.0,8846.0,7679.0
996,45,20210218,495044,4257415.0,1137.0,19701.0,NaN
997,46,20210218,110871,304951.0,92.0,6475.0,NaN
998,47,20210218,761301,NaN,1250.0,18175.0,NaN


In [18]:
# Merging output table

factCovid = pd.merge(factCovid_1, factCovid_2, on = 'fips', how = 'outer')
factCovid

,fips,province_state,country_region,confirmed,deaths,recovered,active,date,positive,negative,hospitalizedcurrently,hospitalized,hospitalizeddischarged
0,NaN,Anhui,China,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Beijing,China,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Chongqing,China,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Fujian,China,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Gansu,China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,56.0,NaN,NaN,NaN,NaN,NaN,NaN,20210223.0,53944.0,179431.0,21.0,1373.0,NaN
1996,56.0,NaN,NaN,NaN,NaN,NaN,NaN,20210222.0,53900.0,179431.0,21.0,1372.0,NaN
1997,56.0,NaN,NaN,NaN,NaN,NaN,NaN,20210221.0,53795.0,178648.0,31.0,1369.0,NaN
1998,56.0,NaN,NaN,NaN,NaN,NaN,NaN,20210220.0,53683.0,178648.0,31.0,1367.0,NaN


In [19]:
# Dimension tables

# Regions

dimRegion_1 = engima_jhud[[
    'fips','province_state','country_region', 'latitude','longitude'
]]

dimRegion_2 = us_county[[
    'fips','county','state'
]]

dimRegion = pd.merge(dimRegion_1, dimRegion_2, on = 'fips', how = 'inner')
dimRegion

,fips,province_state,country_region,latitude,longitude,county,state
0,NaN,Anhui,China,31.826,117.226,New York City,New York
1,NaN,Anhui,China,31.826,117.226,Unknown,Rhode Island
2,NaN,Anhui,China,31.826,117.226,New York City,New York
3,NaN,Anhui,China,31.826,117.226,Unknown,Rhode Island
4,NaN,Anhui,China,31.826,117.226,New York City,New York
...,...,...,...,...,...,...,...
17959,6073.0,California,US,32.716,-117.161,San Diego,California
17960,6073.0,California,US,32.716,-117.161,San Diego,California
17961,6073.0,California,US,32.716,-117.161,San Diego,California
17962,6073.0,California,US,32.716,-117.161,San Diego,California


In [20]:
# Hospital

dimHospital =  engima_jhud[[
    'fips','province_state','country_region', 'latitude','longitude'
]]

In [21]:
# Date

dimDate = states_daily[[
    'fips','date'
]]

dimDate['date'] = pd.to_datetime(dimDate['date'], format='%Y%m%d')
dimDate

C:\Users\ngoqu\AppData\Local\Temp\ipykernel_2096\3090037180.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['date'] = pd.to_datetime(dimDate['date'], format='%Y%m%d')


,fips,date
0,2,2021-03-07
1,1,2021-03-07
2,5,2021-03-07
3,60,2021-03-07
4,4,2021-03-07
...,...,...
995,44,2021-02-18
996,45,2021-02-18
997,46,2021-02-18
998,47,2021-02-18


In [22]:
dimDate['year'] = dimDate['date'].dt.year
dimDate['month'] = dimDate['date'].dt.month
dimDate['day_of_week'] = dimDate['date'].dt.dayofweek

dimDate.head()


C:\Users\ngoqu\AppData\Local\Temp\ipykernel_2096\992337831.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['year'] = dimDate['date'].dt.year
C:\Users\ngoqu\AppData\Local\Temp\ipykernel_2096\992337831.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['month'] = dimDate['date'].dt.month
C:\Users\ngoqu\AppData\Local\Temp\ipykernel_2096\992337831.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

,fips,date,year,month,day_of_week
0,2,2021-03-07,2021,3,6
1,1,2021-03-07,2021,3,6
2,5,2021-03-07,2021,3,6
3,60,2021-03-07,2021,3,6
4,4,2021-03-07,2021,3,6


In [29]:
# Store the final output in S3

# Fact table
factCovid.to_csv(
    f's3://pngo-covid-project/output/factCovid.csv',
    index=False,
    storage_options={
        "key": AWS_ACCESS_KEY,
        "secret": AWS_SECRET_KEY,
        # "token": AWS_SESSION_TOKEN,
    },
)

In [30]:
# Dimension Tables:

dimRegion.to_csv(
    f's3://pngo-covid-project/output/dimRegion.csv',
    index=False,
    storage_options={
        "key": AWS_ACCESS_KEY,
        "secret": AWS_SECRET_KEY,
        # "token": AWS_SESSION_TOKEN,
    },
)

dimHospital.to_csv(
    f's3://pngo-covid-project/output/dimHospital.csv',
    index=False,
    storage_options={
        "key": AWS_ACCESS_KEY,
        "secret": AWS_SECRET_KEY,
        # "token": AWS_SESSION_TOKEN,
    },
)

dimDate.to_csv(
    f's3://pngo-covid-project/output/dimDate.csv',
    index=False,
    storage_options={
        "key": AWS_ACCESS_KEY,
        "secret": AWS_SECRET_KEY,
        # "token": AWS_SESSION_TOKEN,
    },
)

In [33]:
factCovidsql = pd.io.sql.get_schema(factCovid.reindex(), 'factCovid')
print(''.join(factCovidsql))

CREATE TABLE "factCovid" (
"fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "confirmed" REAL,
  "deaths" REAL,
  "recovered" REAL,
  "active" REAL,
  "date" REAL,
  "positive" REAL,
  "negative" REAL,
  "hospitalizedcurrently" REAL,
  "hospitalized" REAL,
  "hospitalizeddischarged" REAL
)


In [34]:
dimDatesql = pd.io.sql.get_schema(dimDate.reindex(), 'dimDate')
print(''.join(dimDatesql))

CREATE TABLE "dimDate" (
"fips" INTEGER,
  "date" TIMESTAMP,
  "year" INTEGER,
  "month" INTEGER,
  "day_of_week" INTEGER
)


In [35]:
dimHospitalsql = pd.io.sql.get_schema(dimHospital.reindex(), 'dimHospital')
print(''.join(dimHospitalsql))

CREATE TABLE "dimHospital" (
"fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "latitude" REAL,
  "longitude" REAL
)


In [36]:
dimRegionsql = pd.io.sql.get_schema(dimRegion.reindex(), 'dimRegionsql')
print(''.join(dimRegionsql))

CREATE TABLE "dimRegionsql" (
"fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "latitude" REAL,
  "longitude" REAL,
  "county" TEXT,
  "state" TEXT
)
